<a href="https://colab.research.google.com/github/anaferreira744/DE-DP-ADF/blob/main/final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [1]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
    - extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [2]:
%pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.9 MB/s eta 0:00:00


# Producer

In [3]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("/content/lake/bronze/messages/data") #step1

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.outputMode('append')
.trigger(processingTime='1 seconds')
.option("checkpointLocation", "/content/lake/bronze/messages/checkpoint") #step1
.foreachBatch(insert_messages)
.start()
)


query.awaitTermination(60)


False

In [4]:
query.isActive

True

In [5]:
query.stop()

In [6]:
df = spark.read.format("parquet").load("/content/lake/bronze/messages/data/*")
df.show()
df.count()

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2024-12-15 14:26:...|    1|      NONE|786ec035-65f3-45d...|   PUSH|      2009|   1013|
|2024-12-15 14:26:...|    3|      NONE|786ec035-65f3-45d...|   PUSH|      2009|   1013|
|2024-12-15 14:26:...|    5|      NONE|786ec035-65f3-45d...|   PUSH|      2009|   1013|
|2024-12-15 14:26:...|    0|      NONE|786ec035-65f3-45d...|   PUSH|      2009|   1013|
|2024-12-15 14:26:...|    2|      NONE|786ec035-65f3-45d...|   PUSH|      2009|   1013|
|2024-12-15 14:26:...|    4|      NONE|786ec035-65f3-45d...|   PUSH|      2009|   1013|
|2024-12-15 14:26:...|   13|  RECEIVED|222831dd-79cc-4d5...|  EMAIL|      2007|   1023|
|2024-12-15 14:26:...|   46|  RECEIVED|786ec035-65f3-45d...|  EMAIL|      2002|   1028|
|2024-12-15 14:26:...|   37|  RE

71

# Additional datasets

In [7]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

# Streaming Messages x Messages Corrupted

In [8]:
df.schema

StructType([StructField('timestamp', TimestampType(), True), StructField('value', LongType(), True), StructField('event_type', StringType(), True), StructField('message_id', StringType(), True), StructField('channel', StringType(), True), StructField('country_id', IntegerType(), True), StructField('user_id', IntegerType(), True)])

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, TimestampType
from pyspark.sql import functions as F
from datetime import datetime


# defining the schema
schema = StructType([
    StructField('timestamp', TimestampType(), True),
    StructField('value', LongType(), True),
    StructField('event_type', StringType(), True),
    StructField('message_id', StringType(), True),
    StructField('channel', StringType(), True),
    StructField('country_id', IntegerType(), True),
    StructField('user_id', IntegerType(), True)
    ])


# Function to write data to two different paths (valid data and corrupted data)
def split(df: DataFrame, batch_id):
    print(f"[INFO] Processing batch {batch_id} at {datetime.now()}")
    print(f"[INFO] Total records in batch: {df.count()}")

    df=df.join(countries, on="country_id", how="left")

    # Corrupted data:
    corrupted_df = df.filter(
        (F.col("event_type").isNull()) |
        (F.col("event_type") == "") |
        (F.col("event_type") == "NONE")
    )

    # Write corrupted data
    corrupted_df.write.mode("append").format("parquet") \
        .partitionBy("date") \
        .save("/content/lake/silver/messages_corrupted/data")
    print(f"[INFO] Corrupted data written successfully for batch {batch_id}.")


    # Valid Data
    valid_df = df.filter(~(
        (F.col("event_type").isNull()) |
        (F.col("event_type") == "") |
        (F.col("event_type") == "NONE")
    ))

    # Write valid data
    valid_df.write.mode("append").format("parquet") \
        .partitionBy("date") \
        .save("/content/lake/silver/messages/data")
    print(f"[INFO] Valid data written successfully for batch {batch_id}.")


# Read the streaming data from bronze
df_stream = spark.readStream.format("parquet") \
    .schema(schema) \
    .load("/content/lake/bronze/messages/data/*")

# Extract date from timestamp
df_enriched = df_stream.withColumn("date", F.to_date("timestamp"))

# Streaming configuration to split the data and write it to the appropriate paths
query = (df_enriched.writeStream
    .outputMode("append")
    .trigger(processingTime="5 seconds")
    .option("checkpointLocation", "/content/lake/silver/checkpoint")
    .foreachBatch(split)
    .start()
)


query.awaitTermination(20)



False

In [12]:
query.isActive

True

In [13]:
query.stop()

## Checking data

In [14]:
# Count messages in Bronze Layer
bronze_df = spark.read.parquet("/content/lake/bronze/messages/data/*")
bronze_count = bronze_df.count()

# Count messages in Silver Layer (valid and corrupted data)
valid_messages_df = spark.read.parquet("/content/lake/silver/messages/data")
corrupted_messages_df = spark.read.parquet("/content/lake/silver/messages_corrupted/data")

valid_messages_count = valid_messages_df.count()
corrupted_messages_count = corrupted_messages_df.count()

# Calculate total messages in Silver Layer (valid + corrupted)
silver_total_count = valid_messages_count + corrupted_messages_count

# Perform the check
if bronze_count == silver_total_count:
    print("[INFO] The counts match: Bronze layer count is equal to the sum of valid and corrupted messages in the Silver layer.")
else:
    print("[ERROR] The counts do not match!")
    print(f"Bronze Layer Count: {bronze_count}")
    print(f"Silver Layer Count (Valid + Corrupted): {silver_total_count}")

[INFO] The counts match: Bronze layer count is equal to the sum of valid and corrupted messages in the Silver layer.


# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

In [15]:
# dedup data
from pyspark.sql import functions as F
from pyspark.sql.window import Window
df = spark.read.format("parquet").load("/content/lake/silver/messages")
dedup = df.withColumn("row_number", F.row_number().over(Window.partitionBy("message_id", "event_type", "channel").orderBy("timestamp"))).filter("row_number = 1").drop("row_number")

### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [16]:
aggregated_df = df.groupBy("date", "event_type", "channel").agg(
    F.count("*").alias("message_count"))

pivoted_df = aggregated_df.groupBy("date", "channel").pivot("event_type").agg(
    F.sum("message_count").alias("message_count"))

pivoted_df.show()

+----------+-------+-------+-------+----+--------+----+
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-15|   PUSH|      1|      2|   1|       4|NULL|
|2024-12-15|  OTHER|      2|   NULL|   2|       7|   1|
|2024-12-15|  EMAIL|      2|      3|   2|       2|   1|
|2024-12-15|    SMS|      2|      1|   1|       1|   2|
|2024-12-15|   CHAT|      1|      3|   6|    NULL|   1|
+----------+-------+-------+-------+----+--------+----+



## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [17]:
from pyspark.sql import functions as F

user_activity_df = df.groupBy("user_id", "channel").agg(F.count("*").alias("iterations"))

pivot_df = user_activity_df.groupBy("user_id").pivot("channel", ["CHAT", "EMAIL", "OTHER", "PUSH", "SMS"]).agg(F.sum("iterations").alias("iterations"))

pivot_df = pivot_df.fillna(0)

pivot_df = pivot_df.withColumn("total_iterations",F.col("CHAT") + F.col("EMAIL") + F.col("OTHER") + F.col("PUSH") + F.col("SMS"))

pivot_df = pivot_df.select("user_id", "total_iterations", "CHAT", "EMAIL", "OTHER", "PUSH", "SMS")

sorted_df = pivot_df.orderBy(F.col("total_iterations"), ascending=False)

sorted_df.show()


+-------+----------------+----+-----+-----+----+---+
|user_id|total_iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------------+----+-----+-----+----+---+
|   1002|               4|   0|    0|    1|   1|  2|
|   1032|               3|   0|    0|    3|   0|  0|
|   1035|               3|   1|    0|    2|   0|  0|
|   1000|               3|   2|    0|    1|   0|  0|
|   1038|               3|   2|    0|    1|   0|  0|
|   1014|               3|   1|    1|    0|   1|  0|
|   1013|               3|   1|    1|    0|   1|  0|
|   1025|               2|   0|    0|    1|   0|  1|
|   1010|               2|   0|    1|    0|   0|  1|
|   1048|               2|   0|    0|    1|   0|  1|
|   1017|               2|   0|    1|    0|   1|  0|
|   1043|               2|   0|    0|    1|   0|  1|
|   1004|               2|   2|    0|    0|   0|  0|
|   1005|               1|   0|    0|    0|   1|  0|
|   1030|               1|   0|    1|    0|   0|  0|
|   1026|               1|   0|    0|    0|   

# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

- Which storage would you use and why? (database?, data lake?, kafka?)




R:
To achieve near real-time message aggregation with low latency using Spark Structured Streaming, we can use Kafka as the data source to stream messages into Spark. Spark will process and aggregate the data in real-time, applying aggregation and time-window operations. Kafka provides high throughput and fault tolerance, allowing continuous data ingestion.
The aggregated results can be stored in a database for fast, low-latency access by the dashboard.